In [25]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-le

In [26]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-21 10:26:58--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-12-21 10:26:59 (6.02 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [27]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData_Amazon_Reviews").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [28]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

amazon_url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/"

# Dataset 2 --> Digital Music
dataset2 = "amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"

amazon_url2 = amazon_url + dataset2
spark.sparkContext.addFile(amazon_url2)
digital_music_df = spark.read.csv(SparkFiles.get(dataset2), inferSchema=True, sep="\t", header=True)

# Show DataFrame
digital_music_df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+----------------------------------------------------------------------------+----------------------+-----------+-------------+-----------+----+-----------------+--------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|marketplace|customer_id|review_id     |product_id|product_parent|product_title                                                               |product_category      |star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline                                         |review_body                                                                                                                                    

In [29]:
# Count the number of records(rows) in the complete dataset.
print('Number of records in Digital music dataset:', digital_music_df.count())

Number of records in Digital music dataset: 1688884


In [30]:
# Transform the dataset to fit the table review_id_table
from pyspark.sql.types import DateType

review_id_df = digital_music_df.select('review_id', 'customer_id', 'product_id', 'product_parent', 'review_date')
review_id_table = review_id_df.withColumn("review_date", review_id_df["review_date"].cast(DateType()))

print('Number of rows in review_id_table:' + str(review_id_table.count()))
review_id_table.orderBy('review_id').show(truncate=False)

Number of rows in review_id_table:1688884
+--------------+-----------+----------+--------------+-----------+
|review_id     |customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100007TERQ36I|20227669   |B002NIWBBI|430027531     |2015-07-22 |
|R10001PSW1P9MN|43072842   |B000TEVKF6|584630580     |2012-12-10 |
|R10002GHQRXG9Q|33776241   |B00IOMX5L2|501710886     |2014-06-05 |
|R10008N8QVGXBM|38876196   |B003Y3XPOI|738771843     |2011-08-11 |
|R10009MVS1J3KE|16982254   |B00137II62|220306261     |2014-04-19 |
|R10009VHWXV2UL|45142846   |B00LQ8IGVK|899670131     |2014-10-04 |
|R1000AG0XB09WV|43615244   |B003DSCGDK|989074968     |2012-11-26 |
|R1000ITBDIA86G|49162427   |B00RH8A288|774112745     |2015-01-02 |
|R1000ITUAN6V4T|45694879   |B00137O9Q0|444308422     |2013-08-13 |
|R1000NV6DPD7Y0|18017303   |B000QP4L5Y|258225989     |2014-09-13 |
|R1000OD0LQNA9M|26278105   |B00GN594QW|739402550     |2013-11-13 |
|R1000RLK5WR40Z|1393

In [31]:
# 'review_id_df' Column list and its data types
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [32]:
# Transform the dataset to fit the table 'products' and table count
products_df = digital_music_df.select('product_id', 'product_title').distinct()
print("Distinct count of products:", products_df.count())
products_df.orderBy('product_id').show(truncate=False)

Distinct count of products: 701548
+----------+------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                               |
+----------+------------------------------------------------------------------------------------------------------------+
|B000PI8FDQ|Lost Together                                                                                               |
|B000PIAUEI|Homegrown                                                                                                   |
|B000PICA8W|Try                                                                                                         |
|B000PIDSQ0|Now and Forever                                                                                             |
|B000PIG5R4|Haiku                                                                                              

In [33]:
# 'products_df' Column list and its data types
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [34]:
# Transform the dataset to fit the 'customer' table and print the count of records
customersCounts = digital_music_df.select('customer_id').groupBy('customer_id').count()
customers_df = customersCounts.withColumnRenamed('count', 'customer_count')

print('Number of records in customers: '+ str(customers_df.count()))
customers_df.orderBy('customer_id').show(truncate=False)

Number of records in customers: 798864
+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|10128      |1             |
|10145      |1             |
|10156      |1             |
|10167      |1             |
|10348      |1             |
|10362      |1             |
|10408      |1             |
|10468      |1             |
|10604      |1             |
|10637      |1             |
|10675      |1             |
|10862      |3             |
|10889      |1             |
|10933      |1             |
|10935      |1             |
|10970      |1             |
|10981      |36            |
|11033      |1             |
|11121      |1             |
|11125      |9             |
+-----------+--------------+
only showing top 20 rows



In [35]:
# 'customers' Table column list and its data types
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [36]:
# Transform the dataset to fit 'vine_table'
vine_table= digital_music_df.select('review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine')
print('Count of records in Vine dataset:' + str(vine_table.count()))
print('Count of records in Vine dataset:' + str(vine_table.distinct().count()))
vine_table.orderBy('review_id').show(truncate=False)

Count of records in Vine dataset:1688884
Count of records in Vine dataset:1688884
+--------------+-----------+-------------+-----------+----+
|review_id     |star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100007TERQ36I|5          |0            |0          |N   |
|R10001PSW1P9MN|5          |0            |0          |N   |
|R10002GHQRXG9Q|5          |0            |0          |N   |
|R10008N8QVGXBM|5          |0            |0          |N   |
|R10009MVS1J3KE|5          |0            |0          |N   |
|R10009VHWXV2UL|5          |0            |0          |N   |
|R1000AG0XB09WV|5          |0            |0          |N   |
|R1000ITBDIA86G|5          |0            |0          |N   |
|R1000ITUAN6V4T|5          |0            |0          |N   |
|R1000NV6DPD7Y0|4          |0            |0          |N   |
|R1000OD0LQNA9M|5          |0            |0          |N   |
|R1000RLK5WR40Z|5          |0            |0          |N   |
|R10013Y9ZSIHO2|5 

In [37]:
# 'vine_table' column list and its data types
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [38]:
#Create the Database properties

mode = "append"
db_url = "jdbc:postgresql://<AWS RDS endpoint>:5432/postgres"

config = {
    "user" : "<username>", 
    "password" : "<password>",
    "driver" : "org.postgresql.Driver"
}

In [39]:
# Load 'review_id_df' dataframe into 'review_id_table

review_id_df.write.jdbc(url=db_url, table='review_id_table', mode=mode, properties=config)

In [40]:
# Load dataframe 'products_df' into RDS Table 'products'

products_df.write.jdbc(url=db_url, table='products', mode=mode, properties=config)

In [41]:
# Load dataframe 'customers_df' into RDS Table 'customers'

customers_df.write.jdbc(url=db_url, table='customers', mode=mode, properties=config)

In [42]:
# Load dataframe 'vine_table' into RDS Table 'vine_table'

vine_table.write.jdbc(url=db_url, table='vine_table', mode=mode, properties=config)